# Problem 6.4: Classifying the Expansion of the Universe Using SNe Data
As was seen in lecture 2, the condition for which the universe's expansion is accelerating was that $w<-1/3$.

Recall that the equation of state is given as,
$$
w = P/\rho,
$$
where $P$ is the pressure, $\rho$ is the density, and $w$ which is the dimensionless equation of state parameter (here, we are working in units where $c=1$). Recall further that for matter, $w_{\rm m} = 0$ whereas for the cosmological constant, $w_\Lambda = -1$. This implies the following relations,
$$
P_\Lambda = -\rho_\Lambda, \qquad P_{\rm m} = 0.
$$
Since we are working in the late-universe, let us assume that *only* matter and the cosmological constants are contibuting components to any form of cosmic expansion, i.e. $\Omega_K \approx \Omega_R \approx 0$. This further implies that $\Omega_\Lambda = 1 - \Omega_{\rm m}$.

Now, for the *total* pressure, we can write,
$$
P_{\rm tot} = P_{\rm m} + P_\Lambda,
$$
however, using the relations above for the pressures, we can write
$$
P_{\rm tot} = P_\Lambda = -\rho_\Lambda,
$$
where we can finally use the relationship that $\rho_\Lambda = \Omega_\Lambda \rho_{\rm crit} = (1-\Omega_{\rm m}) \rho_{\rm crit}$. Therefore, we combine all of this and have,
$$
P_{\rm tot} = -(1-\Omega_{\rm m}) \rho_{\rm crit}.
$$
Next, we wish to derive some form of relationship for $\rho_{\rm tot}$,
$$
\rho_{\rm tot} = \rho_{\rm m}(z) + \rho_\Lambda.
$$
Recalling the above relationship between $\rho_\Lambda$ and $\rho_{\rm crit}$ and again from lecture 2 that $\rho_{\rm m}(z) = \Omega_{\rm m}\rho_{\rm crit}(1+z)^3$, we directly get that
$$
\rho_{\rm tot} = \Omega_{\rm m}\rho_{\rm crit}(1+z)^3 + (1-\Omega_{\rm m})\rho_{\rm crit}.
$$
Therefore, using this, the above relationship we derived for $P_{\rm tot}$ and the relation between these two and $w$, we can calculate $w_{\rm tot}$ given $\Omega_m$.

Your goal in this Colab is to create ML models which take luminosity distances as an input and classify whether or not that data corresponds to an accelerating ($w<-1/3$) or decelerating ($w>-1/3$) universe.

a.) Prepare data from SNe, which gives us observational values for $D_L$ as a function of Redshift. We can use `numpy`'s `np.loadtxt` function to automatically load the data from a `.txt` file and store it to an array.

In [1]:
#Import libraries
import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.cosmology import FlatLambdaCDM  # for flat case use this

In [2]:
#The following code allows us to download the correct data files
!pip install gdown

In [3]:
#Load the data
import gdown

# Function to convert the shared link to the correct format
def convert_shared_link(shared_link):
    return shared_link.replace("?usp=share_link", "?usp=sharing")

shared_link = "https://drive.google.com/file/d/1w9wv1PVthVnRBeMyTlQ72nQkrlY81C0T/view?usp=sharing"

# Convert the shared link to the correct format
shared_link_corrected = convert_shared_link(shared_link)

# Extract the file ID from the shared link
file_id = shared_link_corrected.split("/")[-2]

# Destination path to save the downloaded file in the Colab environment
destination_path = "/content/SNe_data.txt"  # You can modify the path as per your preference

# Download the file using gdown
gdown.download(f"https://drive.google.com/uc?id={file_id}", destination_path, quiet=False)

# Load the data
data = np.loadtxt('/content/SNe_data.txt')
D_L_obs = data[:,1]

D_L_obs = np.flip(data[:,1])
print(data[:,0])
print(D_L_obs)

Downloading...
From: https://drive.google.com/uc?id=1w9wv1PVthVnRBeMyTlQ72nQkrlY81C0T
To: /content/SNe_data.txt
100%|██████████| 78.6k/78.6k [00:00<00:00, 7.49MB/s]

[0.01012 0.01038 0.01043 ... 1.8     1.914   2.26   ]
[18058.89485237 12985.87354162 13228.51729122 ...    46.37150266
    49.1146631     46.00247197]


b.) Prepare the theory data for training and testing of our models. First, we want to calculate $w$ from the above data. We will want to do so for various values of $\Omega_{\rm m}$. However, the *only* value for which this matter is the one at present day, i.e. $z=0$. So for each value of $\Omega_{\rm m}$ we need only calculate $P_{\rm tot}$ (why not $\rho_{\rm tot}$?) to calculate $w_{\rm tot}$. Then, we will use that value to create an array of labels determining if that universe's expansion is accelerating or not. On top of that, we will also want to create an array of angular diameter distances, which should go the same as in Notebook 1 of this lecture.

In [6]:
#YOUR CODE HERE#
# Redshift values
z = [ 0.01012, 0.01038, 0.01043, 0.01082, 0.01209, 0.0122, 0.01226, 0.01268, 0.01271, 0.01291, 0.01318, 0.01382, 0.01402, 0.01447, 0.01451, 0.01454, 0.01472, 0.01504, 0.01506, 0.0152, 0.0153, 0.01531, 0.01531, 0.01532, 0.01567, 0.01568, 0.01572, 0.01629, 0.01635, 0.01662, 0.01663, 0.01678, 0.01697, 0.01698, 0.01705, 0.01724, 0.01726, 0.01732, 0.0174, 0.01741, 0.01752, 0.01754, 0.01763, 0.01908, 0.0197, 0.01972, 0.0202, 0.02024, 0.02031, 0.02038, 0.0204, 0.02091, 0.0212, 0.02171, 0.02189, 0.02197, 0.0221, 0.02223, 0.02249, 0.02275, 0.02348, 0.02349, 0.02384, 0.02398, 0.02401, 0.02414, 0.02454, 0.02477, 0.02482, 0.02525, 0.02529, 0.02547, 0.02557, 0.02559, 0.0258, 0.02627, 0.02639, 0.02643, 0.02651, 0.02673, 0.02689, 0.02751, 0.02752, 0.02752, 0.02763, 0.02811, 0.02837, 0.02845, 0.02846, 0.02878, 0.0288, 0.02894, 0.02932, 0.0295, 0.0299, 0.03001, 0.03048, 0.03049, 0.03055, 0.03144, 0.03155, 0.03161, 0.03163, 0.03184, 0.03191, 0.03212, 0.0322, 0.03273, 0.0328, 0.03292, 0.03314, 0.03348, 0.03359, 0.03378, 0.03399, 0.03408, 0.03411, 0.03416, 0.03426, 0.03434, 0.03438, 0.03489, 0.03501, 0.03507, 0.03518, 0.03578, 0.03682, 0.037, 0.03713, 0.0373, 0.03743, 0.03791, 0.038, 0.03827, 0.03837, 0.03863, 0.03984, 0.04028, 0.04067, 0.04093, 0.04093, 0.0415, 0.04269, 0.0431, 0.04341, 0.04437, 0.04437, 0.04486, 0.04582, 0.04597, 0.04651, 0.0471, 0.04713, 0.04714, 0.04819, 0.04949, 0.04976, 0.05035, 0.05067, 0.05092, 0.05322, 0.05344, 0.05359, 0.05418, 0.05458, 0.05495, 0.05573, 0.05581, 0.05684, 0.05715, 0.05721, 0.0589, 0.05895, 0.05948, 0.06119, 0.06441, 0.06455, 0.06481, 0.06559, 0.06641, 0.06845, 0.06877, 0.06888, 0.0706, 0.07063, 0.07141, 0.0718, 0.07369, 0.07515, 0.07523, 0.0755, 0.07877, 0.07883, 0.07976, 0.08141, 0.0822, 0.08254, 0.08282, 0.08504, 0.08543, 0.08784, 0.08806, 0.08848, 0.0896, 0.0906, 0.0919, 0.09227, 0.09391, 0.09401, 0.0963, 0.0994, 0.10018, 0.1004, 0.1022, 0.1023, 0.10264, 0.10288, 0.103, 0.10337, 0.10351, 0.1056, 0.10638, 0.10683, 0.1072, 0.1075, 0.10886, 0.1098, 0.11242, 0.11364, 0.11628, 0.11635, 0.11741, 0.11771, 0.11797, 0.11845, 0.1187, 0.1194, 0.11966, 0.11991, 0.12042, 0.1206, 0.12164, 0.1219, 0.1221, 0.12262, 0.12303, 0.12336, 0.12376, 0.1238, 0.12455, 0.12516, 0.12608, 0.12622, 0.12688, 0.12694, 0.12715, 0.12837, 0.12899, 0.12903, 0.12928, 0.1294, 0.13045, 0.13243, 0.1326, 0.13353, 0.13514, 0.1352, 0.1361, 0.13639, 0.13687, 0.13696, 0.13729, 0.13811, 0.13818, 0.13833, 0.1384, 0.13864, 0.13868, 0.1388, 0.1396, 0.1407, 0.1412, 0.1431, 0.1436, 0.14387, 0.14409, 0.14478, 0.1453, 0.14533, 0.14581, 0.1466, 0.14703, 0.14729, 0.1476, 0.14792, 0.14882, 0.1496, 0.1505, 0.1506, 0.15186, 0.1526, 0.15271, 0.15304, 0.15317, 0.15325, 0.15339, 0.15492, 0.15509, 0.15517, 0.15596, 0.15653, 0.1568, 0.15689, 0.15837, 0.15899, 0.1592, 0.15935, 0.1599, 0.15999, 0.16035, 0.1604, 0.16059, 0.1612, 0.1618, 0.16284, 0.16356, 0.16431, 0.16455, 0.16477, 0.16595, 0.16817, 0.1692, 0.1696, 0.17028, 0.17063, 0.1712, 0.1717, 0.1727, 0.17315, 0.1735, 0.17357, 0.17373, 0.17383, 0.174, 0.17428, 0.17483, 0.17558, 0.17625, 0.17653, 0.17696, 0.17706, 0.17768, 0.1777, 0.17789, 0.17839, 0.17851, 0.1789, 0.17915, 0.1792, 0.1792, 0.17958, 0.1799, 0.18037, 0.1806, 0.1806, 0.18079, 0.1808, 0.1808, 0.1812, 0.18123, 0.18203, 0.1828, 0.18322, 0.18376, 0.18388, 0.18443, 0.18468, 0.18481, 0.185, 0.18507, 0.18516, 0.1852, 0.18563, 0.18581, 0.18651, 0.18891, 0.18914, 0.18931, 0.1896, 0.18979, 0.19009, 0.19025, 0.19029, 0.1912, 0.19176, 0.19392, 0.19422, 0.1958, 0.1959, 0.19611, 0.19654, 0.19671, 0.19718, 0.19726, 0.19801, 0.1985, 0.19862, 0.19867, 0.1994, 0.1996, 0.1998, 0.2006, 0.2006, 0.20215, 0.20267, 0.20308, 0.20313, 0.20318, 0.20323, 0.20335, 0.20368, 0.20453, 0.2049, 0.20515, 0.20548, 0.20557, 0.20735, 0.20847, 0.2085, 0.20852, 0.2088, 0.21005, 0.2102, 0.2102, 0.21054, 0.2106, 0.21075, 0.21082, 0.21099, 0.2116, 0.21163, 0.21179, 0.21226, 0.21323, 0.21338, 0.21353, 0.2145, 0.21457, 0.21513, 0.2166, 0.21674, 0.21728, 0.21757, 0.21835, 0.21865, 0.21885, 0.21887, 0.21935, 0.21938, 0.21955, 0.2196, 0.21975, 0.22068, 0.2212, 0.2212, 0.22256, 0.22286, 0.22334, 0.2236, 0.22452, 0.2246, 0.2253, 0.22534, 0.2255, 0.22669, 0.22786, 0.2288, 0.22916, 0.2296, 0.22967, 0.2305, 0.2307, 0.2307, 0.2318, 0.2335, 0.2338, 0.23381, 0.2348, 0.23542, 0.2363, 0.23652, 0.2367, 0.2368, 0.23684, 0.23775, 0.2388, 0.2396, 0.24, 0.2406, 0.2411, 0.2416, 0.24185, 0.24206, 0.2426, 0.24267, 0.24315, 0.2434, 0.24436, 0.24448, 0.2448, 0.2454, 0.2454, 0.24552, 0.24555, 0.24558, 0.2457, 0.24603, 0.24637, 0.24639, 0.24643, 0.24835, 0.24852, 0.24857, 0.2488, 0.2488, 0.24883, 0.2492, 0.24961, 0.24973, 0.24973, 0.25, 0.2501, 0.25029, 0.25037, 0.2504, 0.2504, 0.25104, 0.25174, 0.25191, 0.25249, 0.2528, 0.2534, 0.25609, 0.25625, 0.25655, 0.25672, 0.25721, 0.25722, 0.2576, 0.258, 0.2588, 0.2604, 0.2607, 0.2607, 0.26129, 0.26162, 0.26166, 0.2622, 0.2631, 0.26338, 0.26349, 0.26357, 0.2637, 0.26405, 0.2641, 0.26451, 0.2647, 0.26576, 0.26687, 0.268, 0.26821, 0.26838, 0.26856, 0.2688, 0.26891, 0.27, 0.2701, 0.2707, 0.2708, 0.27091, 0.2712, 0.2734, 0.2748, 0.2749, 0.2752, 0.27544, 0.27573, 0.27656, 0.27749, 0.27822, 0.27923, 0.2797, 0.2798, 0.2804, 0.2824, 0.28332, 0.28391, 0.2842, 0.28434, 0.28436, 0.2854, 0.28543, 0.2856, 0.2858, 0.28655, 0.28662, 0.28742, 0.28746, 0.2888, 0.2889, 0.2896, 0.2902, 0.29021, 0.2907, 0.29114, 0.29135, 0.2916, 0.2921, 0.2927, 0.2928, 0.2928, 0.29353, 0.2956, 0.29567, 0.29683, 0.2988, 0.2988, 0.29888, 0.29936, 0.2996, 0.3001, 0.30021, 0.30041, 0.3006, 0.3012, 0.3018, 0.30191, 0.3026, 0.3032, 0.30354, 0.30394, 0.3051, 0.3055, 0.3056, 0.30569, 0.30632, 0.3065, 0.3068, 0.308, 0.30915, 0.3092, 0.30929, 0.30938, 0.3106, 0.31065, 0.3112, 0.31288, 0.31376, 0.31468, 0.3148, 0.31494, 0.3156, 0.31582, 0.318, 0.3185, 0.3188, 0.3192, 0.3192, 0.32045, 0.3207, 0.3212, 0.3218, 0.32416, 0.3253, 0.3254, 0.32569, 0.3264, 0.32793, 0.3283, 0.3288, 0.3288, 0.32931, 0.3297, 0.32984, 0.3302, 0.33051, 0.3306, 0.3306, 0.33103, 0.3312, 0.3312, 0.3312, 0.33136, 0.3316, 0.3327, 0.33439, 0.3347, 0.3374, 0.3374, 0.3376, 0.3379, 0.3386, 0.3388, 0.34039, 0.3412, 0.3412, 0.3456, 0.34621, 0.347, 0.3473, 0.349, 0.349, 0.3491, 0.34916, 0.34916, 0.3492, 0.3503, 0.3509, 0.35116, 0.3512, 0.35346, 0.3536, 0.35486, 0.35568, 0.35816, 0.35819, 0.359, 0.36, 0.36, 0.3606, 0.3618, 0.36193, 0.3652, 0.3666, 0.3669, 0.3676, 0.3682, 0.36952, 0.36976, 0.3701, 0.37016, 0.37039, 0.3704, 0.37046, 0.37091, 0.3712, 0.3712, 0.37129, 0.37191, 0.37391, 0.37425, 0.3782, 0.37986, 0.3802, 0.3804, 0.38048, 0.3806, 0.3812, 0.3815, 0.3826, 0.38492, 0.3857, 0.3876, 0.3878, 0.3892, 0.3922, 0.40091, 0.40127, 0.40128, 0.4019, 0.40439, 0.4045, 0.40591, 0.4096, 0.4098, 0.40991, 0.4114, 0.4118, 0.41616, 0.41816, 0.4194, 0.4198, 0.4198, 0.4206, 0.4208, 0.42291, 0.4231, 0.4235, 0.4267, 0.4269, 0.4276, 0.42816, 0.4292, 0.4294, 0.4305, 0.43521, 0.43591, 0.4365, 0.4391, 0.4392, 0.4399, 0.4404, 0.44238, 0.44316, 0.4474, 0.44939, 0.4504, 0.4507, 0.45117, 0.4512, 0.45138, 0.4604, 0.4607, 0.46109, 0.46138, 0.4661, 0.46691, 0.46891, 0.47039, 0.4704, 0.47091, 0.4712, 0.47516, 0.4774, 0.4796, 0.48016, 0.48038, 0.4804, 0.4812, 0.4892, 0.4952, 0.49791, 0.50016, 0.5006, 0.5025, 0.5027, 0.50349, 0.5078, 0.50791, 0.5088, 0.51, 0.5106, 0.51116, 0.51416, 0.5142, 0.51491, 0.51539, 0.5192, 0.5194, 0.5196, 0.52216, 0.5283, 0.53216, 0.53316, 0.53491, 0.53516, 0.53591, 0.545, 0.54891, 0.5501, 0.55091, 0.55239, 0.55316, 0.55819, 0.5592, 0.5592, 0.5632, 0.5652, 0.57516, 0.5761, 0.57639, 0.5779, 0.57891, 0.57921, 0.57938, 0.58038, 0.58119, 0.5834, 0.58392, 0.58421, 0.58421, 0.58621, 0.58791, 0.5892, 0.59091, 0.59116, 0.60038, 0.60392, 0.60891, 0.60916, 0.6104, 0.61191, 0.6162, 0.6192, 0.62039, 0.62116, 0.62591, 0.62791, 0.6304, 0.63116, 0.6312, 0.63291, 0.63821, 0.63892, 0.64116, 0.64339, 0.64339, 0.64739, 0.64839, 0.66259, 0.66438, 0.67039, 0.6782, 0.68116, 0.68339, 0.68591, 0.68921, 0.69038, 0.69391, 0.69791, 0.69892, 0.69916, 0.69992, 0.69992, 0.70116, 0.70116, 0.7012, 0.70191, 0.71538, 0.71839, 0.7202, 0.72039, 0.72091, 0.72639, 0.72739, 0.73091, 0.73316, 0.73416, 0.7342, 0.735, 0.73516, 0.7362, 0.7364, 0.74, 0.74116, 0.7424, 0.74526, 0.74538, 0.74991, 0.75039, 0.75639, 0.75816, 0.76039, 0.76191, 0.7622, 0.7662, 0.76639, 0.7672, 0.76791, 0.7692, 0.77391, 0.77721, 0.78891, 0.78991, 0.7972, 0.7992, 0.80039, 0.80538, 0.80538, 0.80691, 0.80892, 0.80991, 0.8104, 0.81338, 0.8174, 0.82116, 0.82238, 0.8292, 0.83716, 0.83791, 0.84091, 0.84116, 0.8412, 0.8489, 0.8492, 0.85039, 0.85391, 0.854, 0.8592, 0.8592, 0.8592, 0.86491, 0.8652, 0.8672, 0.87116, 0.87891, 0.89038, 0.89216, 0.89791, 0.90139, 0.91039, 0.9142, 0.92116, 0.92116, 0.92516, 0.92591, 0.92891, 0.93116, 0.93291, 0.93491, 0.935, 0.93639, 0.94791, 0.94921, 0.95039, 0.95991, 0.96038, 0.96038, 0.97, 0.9752, 0.98216, 0.98339, 0.9842, 0.9972, 0.99891, 1.0024, 1.014, 1.02, 1.02991, 1.06039, 1.092, 1.12, 1.206, 1.23, 1.23, 1.3, 1.305, 1.315, 1.33, 1.34, 1.37, 1.39, 1.54, 1.55, 1.7, 1.8, 1.914, 2.26 ]

OmM = np.arange(0.1,0.9 + 0.01,0.01)
OmL = 1 - OmM

H = np.arange(50,80 + 0.1,0.1)

In [7]:
lum_dist = []
check_acc = []
for i in range(len(OmM)):
  for j in range(len(H)):
    cosmo = FlatLambdaCDM(H0=H[j], Om0=OmM[i])
    lum_dist.append(cosmo.luminosity_distance(z))
    if OmL[i] > 1/3:
      check_acc.append(0)
    else: check_acc.append(1)

In [8]:
X = lum_dist
y = np.array(np.meshgrid(OmM, H)).T.reshape(-1,2)

c.) Prepare the data for use in our ML algorithms. Make sure all labels assume values such as 0 or 1 rather than strings. Then split the data into training and testing data sets.

In [ ]:
# Create a dictionary to map class labels to numerical labels
translate_dict = {'Accelerated': 0, 'Decelerated': 1}
inverse_translate_dict = {v: k for k, v in translate_dict.items()}

# Map class labels to numerical labels
#numerical_labels = np.array([translate_dict[label] for label in labels])

In [9]:
from sklearn.model_selection import train_test_split
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

d.) Let's train and predict! We will use a decision tree classifier, random forest classifier, and KNN classifier.

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
# DecisionTreeClassifier
# Training
# DecisionTreeClassifier
my_tree = DecisionTreeClassifier(max_depth=4,random_state=1)

my_tree.fit(X_train, y_train)

# Use model to 'predict' the test data
predictions = my_tree.predict(D_L_obs.reshape(1, -1))
# Use the accuracy metric to determine how accurate the model is
accuracy = accuracy_score(y_test, predictions)
print("Accuracy (DecisionTree):", accuracy)

ValueError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# RandomForestClassifier
# Training
#YOUR CODE HERE#

# Evaluate the classifier
#YOUR CODE HERE#

# Use model to predict acceleration from BAO data
#YOUR CODE HERE#

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# KNNClassifier
# Training
#YOUR CODE HERE#

# Evaluate the classifier
#YOUR CODE HERE#

# Use model to predict acceleration from BAO data
#YOUR CODE HERE#